# P4 - Human in the loop

本节重点： 
- 在实际工作中，我们都必须要有人来把关AI输出的结果，确保其准确性。

In [17]:
from typing import Annotated

from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


memory = MemorySaver()


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


tool = TavilySearchResults(max_results=2)
tools = [tool]
llm = ChatOpenAI(model="gpt-4o")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)


tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges("chatbot", tools_condition,)
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")




In [18]:

# from IPython.display import Image, display
# display(Image(graph.get_graph().draw_mermaid_png()))



# Improved - Ask user to continue or not.

In [ ]:
graph = graph_builder.compile(
    checkpointer=memory,
    interrupt_before=["tools"],
    # Note: can also interrupt __after__ tools, if desired.
    # interrupt_after=["tools"]
)

from typing import Union

def handle_interrupt() -> Union[bool, None]:
    user_choice = input("Would you like to continue? (yes/no): ")
    if user_choice.lower().strip().startswith('y'):
        return True
    return False

config = {"configurable": {"thread_id": "1"}}

while True:
    user_input = input("Please enter your input: ")

    events = graph.stream(
        {"messages": [("user", user_input)]}, config, stream_mode="values"
    )

    for event in events:
        if "messages" in event:
            event["messages"][-1].pretty_print()

    snapshot = graph.get_state(config)
    if snapshot.next == ('tools',):
        interrupted = True
        should_continue = handle_interrupt()
        if not should_continue:
            print("\n=== Process aborted by user ===")
            break
    else:
        interrupted = False

    if not interrupted:
        break

    # Continue the graph
    events = graph.stream(None, config, stream_mode="values")
    for event in events:
        if "messages" in event:
            event["messages"][-1].pretty_print()

    # Check again for interruption
    snapshot = graph.get_state(config)
    if snapshot.next == ('tools',):
        interrupted = True
        should_continue = handle_interrupt()
        if not should_continue:
            print("\n=== Process aborted by user ===")
            break
    else:
        interrupted = False

    if not interrupted:
        break

================================ Human Message =================================

what's the latest news?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_Bv75ecQP53vVqe7KlPpvw3NR)
 Call ID: call_Bv75ecQP53vVqe7KlPpvw3NR
  Args:
    query: latest news

=== Process Interrupted ===
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_Bv75ecQP53vVqe7KlPpvw3NR)
 Call ID: call_Bv75ecQP53vVqe7KlPpvw3NR
  Args:
    query: latest news
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://apnews.com/", "content": "In a political shift to the far right, anti-Islam populist Geert Wilders wins big in Dutch election\nEurope’s far-right populists buoyed by Wilders’ win in Netherlands, hoping the best is yet to come\nDaniel Noboa is sworn in as Ecuador’s president, inheri